In [1]:
import pandas as pd
import datetime
import time
from ckanapi import RemoteCKAN

scidm = RemoteCKAN('https://scidm.nchc.org.tw', apikey='API_KEY')

In [2]:
dataset = pd.read_csv('dataset.csv', header=None, names=['ip', 'dataset','download', 'date', 'size', 'request'])
dataset_org = pd.read_csv('dataset-organization.csv', header=None, names = ['org', 'ip', 'dataset', 'download', 'date', 'size', 'requests'])
dataset_group = pd.read_csv('dataset-group.csv', header=None, names=['group', 'ip', 'dataset','download', 'date', 'size', 'request'])

In [3]:
dataset_org['date'] = pd.to_datetime(dataset_org.date, format = '%d/%b/%Y:%H:%M:%S +0800')
dataset_org.head()

,org,ip,dataset,download,date,size,requests
0,False,66.222.22.250,False,False,2019-03-25 06:25:35,4869,GET / HTTP/1.0
1,False,66.222.22.250,False,False,2019-03-25 06:26:07,4869,GET / HTTP/1.0
2,False,66.222.22.250,False,False,2019-03-25 06:26:18,4869,GET / HTTP/1.0
3,False,66.222.22.250,False,False,2019-03-25 06:26:22,4869,GET / HTTP/1.0
4,False,66.222.22.250,False,False,2019-03-25 06:28:05,4869,GET / HTTP/1.0


In [4]:
dataset_org_date = dataset_org[(dataset_org['date'] > '2018-12') & (dataset_org['date'] < '2019-05')].sort_values(by=['date'])
dataset_org_date.head()

,org,ip,dataset,download,date,size,requests
523100,False,220.136.1.12,False,False,2018-12-01 00:00:28,531,GET /fanstatic/privatedatasets/:version:2018-1...
523099,False,220.136.1.12,False,False,2018-12-01 00:00:28,7044,GET /zh_TW/dataset?q=LASS%E7%92%B0%E5%A2%83%E6...
523101,False,220.136.1.12,False,False,2018-12-01 00:00:29,2947,GET /api/i18n/zh_TW HTTP/1.0
523102,False,66.249.71.15,False,False,2018-12-01 00:01:13,4065,GET /ja/dataset/activity/382000000a-001063 HTT...
523103,ipgod-nchc,66.249.71.15,9280fa96-9efe-4614-a734-4c2e6a78f46e,False,2018-12-01 00:05:17,5587,GET /dataset/382000000a-001946 HTTP/1.0


## 組織被瀏覽統計

In [5]:
org_count = pd.DataFrame({'count' : dataset_org_date.groupby(['org']).size()}).reset_index().sort_values('count', ascending=False)
org_count = org_count[org_count['org'] != 'False']

org_name_list = []
for org_name in list(org_count['org']):
    org_name_list.append(scidm.call_action('organization_show', {'id': org_name})['title'])

org_count['org'] = org_name_list
org_count

,org,count
13,高雄市政府資料開放平台,98019
30,TCIA,77363
1,aiDM,65795
12,政府開放資料彙整平台,62438
5,湖泊與環境監測,52303
3,資料平台計劃專區,32975
2,臺南市資料開放平台,26890
20,整合開放資料於開放市集平台,22203
7,GitHub Archive,21328
27,各地郵政,20826


In [6]:
org_count.to_csv('201812_201904組織被瀏覽統計.csv', index = False)

## 組織被下載統計

In [8]:
org_count_download = dataset_org_date[~dataset_org_date['download'] == False]
org_count_download.head()

,org,ip,dataset,download,date,size,requests
523127,environment-quality,140.110.20.153,7a23dd3e-642c-4a0e-8cb9-3793ff64948b,True,2018-12-01 00:10:10,8719,GET /dataset/7a23dd3e-642c-4a0e-8cb9-3793ff649...
523183,ipgod-nchc,59.124.201.21,abd39dec-06ea-4958-ae3a-343a80606368,True,2018-12-01 00:23:32,2230,GET /dataset/a07100000d-000471/resource/533147...
523243,environment-quality,220.136.1.12,7a23dd3e-642c-4a0e-8cb9-3793ff64948b,True,2018-12-01 00:45:12,4036,GET /dataset/7a23dd3e-642c-4a0e-8cb9-3793ff649...
523277,environment-quality,220.136.1.12,7a23dd3e-642c-4a0e-8cb9-3793ff64948b,True,2018-12-01 00:47:17,77651,GET /dataset/7a23dd3e-642c-4a0e-8cb9-3793ff649...
523279,environment-quality,66.249.71.13,b59ac9a5-e943-4a70-9aa5-fe5906baebf6,True,2018-12-01 00:47:25,2028,GET /dataset/yylake/resource/64f2c954-88e3-405...


In [9]:
org_count_download = pd.DataFrame({'count' : org_count_download.groupby(['org']).size()}).reset_index().sort_values('count', ascending=False)
org_count_download = org_count_download[org_count_download['org'] != 'False']

org_name_list = []
for org_name in list(org_count_download['org']):
    org_name_list.append(scidm.call_action('organization_show', {'id': org_name})['title'])

org_count_download['org'] = org_name_list
org_count_download

,org,count
26,TCIA,30281
18,整合開放資料於開放市集平台,13931
11,政府開放資料彙整平台,10650
12,高雄市政府資料開放平台,7750
28,UCI Machine Learning Repository,4893
1,aiDM,4581
5,湖泊與環境監測,4351
16,國家太空中心,3557
9,智慧科學園區,2178
3,資料平台計劃專區,2140


In [10]:
org_count_download.to_csv('201812_201904組織被下載統計.csv', index= False)

## 分類被瀏覽

In [11]:
dataset_group['date'] = pd.to_datetime(dataset_org.date, format = '%d/%b/%Y:%H:%M:%S +0800')
dataset_group.head()

,group,ip,dataset,download,date,size,request
0,research,66.249.71.10,60e0a685-9454-44e7-83e5-09a87264f04e,False,2019-03-25 06:25:35,7612,GET /dataset/nih-clinical-center-deeplesion/re...
1,research,66.249.71.9,60e0a685-9454-44e7-83e5-09a87264f04e,True,2019-03-25 06:26:07,56900,GET /dataset/nih-clinical-center-deeplesion/re...
2,traffic,114.32.56.155,370ccf2c-41dd-4a6d-ac54-e507d6d1529a,False,2019-03-25 06:26:18,5675,GET /dataset/http-www-post-gov-tw-post-interne...
3,publicinformation,114.32.56.155,370ccf2c-41dd-4a6d-ac54-e507d6d1529a,False,2019-03-25 06:26:22,5675,GET /dataset/http-www-post-gov-tw-post-interne...
4,research,104.222.16.198,441de35d-6f55-4894-9f36-e9cf52583024,False,2019-03-25 06:28:05,6217,GET /dataset/detection_of_iot_botnet_attacks_n...


In [12]:
dataset_group_date = dataset_group[(dataset_group['date'] > '2018-12') & (dataset_group['date'] < '2019-05')].sort_values(by=['date'])
dataset_group_date.head()

,group,ip,dataset,download,date,size,request
265156,medical,66.249.71.13,ff832511-9f24-4628-aedb-d7698add0413,True,2018-12-27 06:25:58,2045,GET /dataset/ntuh_ffr/resource/ed03dcb4-10c7-4...
265157,research,66.249.71.14,0080258d-3658-47ee-b2cf-cba8b2f6e72e,False,2018-12-27 06:27:25,7059,GET /dataset/github_archive_2018q3/resource/02...
265158,research,73.103.82.114,61af0e96-0aa3-4a47-97a6-a102de932f0c,False,2018-12-27 06:29:13,5219,GET /dataset/csm-conventional-and-social-media...
265159,medical,66.249.71.13,256c374f-7798-4b75-9ced-887b06b1a554,False,2018-12-27 06:32:33,5294,GET /dataset/ntuh_cag/resource/2a09b490-bd4c-4...
265160,medical,66.249.71.13,256c374f-7798-4b75-9ced-887b06b1a554,True,2018-12-27 06:33:51,4681,GET /dataset/ntuh_cag/resource/2a09b490-bd4c-4...


In [13]:
group_count = pd.DataFrame({'count' : dataset_group_date.groupby(['group']).size()}).reset_index().sort_values('count', ascending=False)

group_name_list = []
for group_name in list(group_count['group']):
#     print(scidm.call_action('group_show', {'id': group_name})['title'])
    group_name_list.append(scidm.call_action('group_show', {'id': group_name})['title'])

group_count['group'] = group_name_list
group_count

,group,count
6,科研,112473
1,環境,40716
3,生活,33707
5,政府治理,28538
4,醫療,27305
7,交通,24264
0,AI-ML,850
8,語音,748
2,資訊安全,650


In [14]:
group_count.to_csv('201812_201904分類被瀏覽統計.csv', index = False)

## 分類被下載統計

In [15]:
group_count_download = dataset_group_date[~dataset_group_date['download'] == False]
group_count_download.head()

,group,ip,dataset,download,date,size,request
265156,medical,66.249.71.13,ff832511-9f24-4628-aedb-d7698add0413,True,2018-12-27 06:25:58,2045,GET /dataset/ntuh_ffr/resource/ed03dcb4-10c7-4...
265160,medical,66.249.71.13,256c374f-7798-4b75-9ced-887b06b1a554,True,2018-12-27 06:33:51,4681,GET /dataset/ntuh_cag/resource/2a09b490-bd4c-4...
265184,environment,66.249.71.14,8406285b-e878-4033-98ea-ad5aa58db9af,True,2018-12-27 06:40:01,1628,GET /dataset/lass-2017q3/resource/b9704f9b-67f...
265185,life,66.249.71.14,8406285b-e878-4033-98ea-ad5aa58db9af,True,2018-12-27 06:40:01,1628,GET /dataset/lass-2017q3/resource/b9704f9b-67f...
265189,life,66.249.71.14,92699b39-0825-4801-8426-5ee00cede691,True,2018-12-27 06:40:01,1632,GET /dataset/lass-2017q2/resource/21a5973c-14d...


In [16]:
group_count_download = pd.DataFrame({'count' : group_count_download.groupby(['group']).size()}).reset_index().sort_values('count', ascending=False)
group_count_download

group_name_list = []
for group_name in list(group_count_download['group']):
    group_name_list.append(scidm.call_action('group_show', {'id': group_name})['title'])

group_count_download['group'] = group_name_list
group_count_download

,group,count
6,科研,18525
1,環境,13177
3,生活,9214
5,政府治理,5950
4,醫療,3349
7,交通,2664
8,語音,261
2,資訊安全,207
0,AI-ML,104


In [17]:
group_count_download.to_csv('201812_201904分類被下載統計.csv', index = False)